In [1]:
import asyncio
import datetime
import ssl
import pickle
import json

In [2]:
class test():
    def __init__(self, a):
        self.a = a
        self.b = 5
        
    def mult(self):
        print(self.a*self.b)

In [3]:
class result():
    def __init__(self, c):
        self.c = c
        
    def disp(self):
        print(f'Result is {self.c}')
        return self.c

In [4]:
async def handle_echo(reader, writer):
    i = 0
    while (i < 10):
        message = await reader.read(100)
        dt = datetime.datetime.now()
        data = pickle.loads(message)
        data = result(data.mult())
        addr = writer.get_extra_info('peername')

        print(f"Time: {dt} Received {message!r} from {addr!r}")
        
        message = pickle.dumps(data)
        dt = datetime.datetime.now()
        print(f"Time: {dt} Send: {message!r}")
        writer.write(message)
        await writer.drain()
        if message == "quit":
            break
        i = i + 1
        await asyncio.sleep(2)

    print("Close the connection")
    writer.close()
    await writer.wait_closed()

In [5]:
async def main():
    ctx = ssl.SSLContext(protocol=ssl.PROTOCOL_TLS_SERVER)
    ctx.load_cert_chain('cpoCertChain.pem', keyfile='seccLeaf.key', password='12345')
    ctx.load_verify_locations(cafile='oemRootCACert.pem')
    ctx.check_hostname = False
    ctx.verify_mode = ssl.VerifyMode.CERT_REQUIRED
    ctx.set_ciphers('ECDHE-ECDSA-AES256-GCM-SHA384:ECDHE-RSA-AES256-GCM-SHA384')
    server = await asyncio.start_server(
        handle_echo, '127.0.0.1', 9002, ssl=ctx)

    addrs = ', '.join(str(sock.getsockname()) for sock in server.sockets)
    print(f'Serving on {addrs}')

    async with server:
        await server.serve_forever()

In [ ]:
await main()

Serving on ('127.0.0.1', 9002)
0
Time: 2023-03-08 21:59:47.212755 Received b'\x80\x04\x95)\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x04test\x94\x93\x94)\x81\x94}\x94(\x8c\x01a\x94K\x00\x8c\x01b\x94K\x05ub.' from ('127.0.0.1', 9921)
Time: 2023-03-08 21:59:47.213778 Send: b'\x80\x04\x95#\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x06result\x94\x93\x94)\x81\x94}\x94\x8c\x01c\x94Nsb.'
